In [1]:
import os
import cv2
import imghdr
import numpy as np
import pathlib
from tensorflow import keras
from keras.models import Sequential
from keras import optimizers
from keras.layers import Convolution2D, MaxPooling2D, Dropout, Flatten, Dense, Reshape, LSTM, BatchNormalization
from keras.optimizers import SGD, RMSprop, Adam
from keras import backend as K
from keras.constraints import maxnorm
import tensorflow as tf
from scipy import io as spio
import idx2numpy  # sudo pip3 install idx2numpy
from matplotlib import pyplot as plt
from typing import *
import time

In [2]:
from tensorflow.compat.v1.keras.backend import get_session

In [3]:
def letters_extract(image_file: str, out_size=28) -> List[Any]:
    img = cv2.imread(image_file)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    ret, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY)
    img_erode = cv2.erode(thresh, np.ones((3, 3), np.uint8), iterations=1)
    contours, hierarchy = cv2.findContours(img_erode, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)

    output = img.copy()
    letters = []
    for idx, contour in enumerate(contours):
        (x, y, w, h) = cv2.boundingRect(contour)
        if hierarchy[0][idx][3] == 0:
            cv2.rectangle(output, (x, y), (x + w, y + h), (70, 0, 0), 1)
            letter_crop = gray[y:y + h, x:x + w]
            size_max = max(w, h)
            letter_square = 255 * np.ones(shape=[size_max, size_max], dtype=np.uint8)
            if w > h:
                y_pos = size_max//2 - h//2
                letter_square[y_pos:y_pos + h, 0:w] = letter_crop
            elif w < h:
                x_pos = size_max//2 - w//2
                letter_square[0:h, x_pos:x_pos + w] = letter_crop
            else:
                letter_square = letter_crop

            letters.append((x, w, cv2.resize(letter_square, (out_size, out_size), interpolation=cv2.INTER_AREA)))
            
    letters.sort(key=lambda x: x[0], reverse=False)

    return letters

In [4]:
letters = letters_extract(image_file)

In [5]:
def emnist_model():
    model = Sequential()
    model.add(Convolution2D(filters=32, kernel_size=(3, 3), padding='valid', input_shape=(28, 28, 1), activation='relu'))
    model.add(Convolution2D(filters=64, kernel_size=(3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(len(emnist_labels), activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adadelta', metrics=['accuracy'])
    return model

In [6]:
emnist_labels = [48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122]

emnist_path = 'C:/Users/niketaqq/emnist/'
X_train = idx2numpy.convert_from_file(emnist_path + 'emnist-byclass-train-images-idx3-ubyte')
y_train = idx2numpy.convert_from_file(emnist_path + 'emnist-byclass-train-labels-idx1-ubyte')

X_test = idx2numpy.convert_from_file(emnist_path + 'emnist-byclass-test-images-idx3-ubyte')
y_test = idx2numpy.convert_from_file(emnist_path + 'emnist-byclass-test-labels-idx1-ubyte')

X_train = np.reshape(X_train, (X_train.shape[0], 28, 28, 1))
X_test = np.reshape(X_test, (X_test.shape[0], 28, 28, 1))

k = 3
X_train = X_train[:X_train.shape[0] // k]
y_train = y_train[:y_train.shape[0] // k]
X_test = X_test[:X_test.shape[0] // k]
y_test = y_test[:y_test.shape[0] // k]

X_train = X_train.astype(np.float32)
X_train /= 255.0
X_test = X_test.astype(np.float32)
X_test /= 255.0

x_train_cat = keras.utils.to_categorical(y_train, len(emnist_labels))
y_test_cat = keras.utils.to_categorical(y_test, len(emnist_labels))

In [7]:
model2 = emnist_model()

In [8]:
learning_rate_reduction = keras.callbacks.ReduceLROnPlateau(monitor='val_acc', patience=3, verbose=1, factor=0.5, min_lr=0.00001)

tf.compat.v1.keras.backend.get_session().run(tf.compat.v1.global_variables_initializer())

model2.fit(X_train, x_train_cat, validation_data=(X_test, y_test_cat), callbacks=[learning_rate_reduction], batch_size=64, epochs=30)

model2.save('emnist_letters2.h5')

Train on 232644 samples, validate on 38774 samples
Epoch 1/30
232644/232644 [==============================] - 344s 1ms/sample - loss: 3.7728 - accuracy: 0.1373 - val_loss: 3.2505 - val_accuracy: 0.3975
Epoch 2/30
232644/232644 [==============================] - 343s 1ms/sample - loss: 2.9870 - accuracy: 0.3348 - val_loss: 2.4303 - val_accuracy: 0.4728
Epoch 3/30
232644/232644 [==============================] - 361s 2ms/sample - loss: 2.4163 - accuracy: 0.4350 - val_loss: 1.9617 - val_accuracy: 0.5401
Epoch 4/30
232644/232644 [==============================] - 345s 1ms/sample - loss: 2.1131 - accuracy: 0.4871 - val_loss: 1.7173 - val_accuracy: 0.5806
Epoch 5/30
232644/232644 [==============================] - 339s 1ms/sample - loss: 1.9355 - accuracy: 0.5171 - val_loss: 1.5676 - val_accuracy: 0.6047
Epoch 6/30
232644/232644 [==============================] - 334s 1ms/sample - loss: 1.8142 - accuracy: 0.5399 - val_loss: 1.4633 - val_accuracy: 0.6221
Epoch 7/30
232644/232644 [===========

232644/232644 [==============================] - 354s 2ms/sample - loss: 1.2880 - accuracy: 0.6454 - val_loss: 0.9994 - val_accuracy: 0.7201
Epoch 22/30
232644/232644 [==============================] - 348s 1ms/sample - loss: 1.2727 - accuracy: 0.6473 - val_loss: 0.9873 - val_accuracy: 0.7233
Epoch 23/30
232644/232644 [==============================] - 349s 2ms/sample - loss: 1.2573 - accuracy: 0.6514 - val_loss: 0.9740 - val_accuracy: 0.7255
Epoch 24/30
232644/232644 [==============================] - 344s 1ms/sample - loss: 1.2448 - accuracy: 0.6544 - val_loss: 0.9640 - val_accuracy: 0.7283
Epoch 25/30
232644/232644 [==============================] - 341s 1ms/sample - loss: 1.2349 - accuracy: 0.6570 - val_loss: 0.9535 - val_accuracy: 0.7305
Epoch 26/30
232644/232644 [==============================] - 352s 2ms/sample - loss: 1.2180 - accuracy: 0.6610 - val_loss: 0.9441 - val_accuracy: 0.7327
Epoch 27/30
232644/232644 [==============================] - 333s 1ms/sample - loss: 1.2106 - 

In [9]:
model = keras.models.load_model('emnist_letters2.h5')

def emnist_predict_img(model, img):
    img_arr = np.expand_dims(img, axis=0)
    img_arr = 1 - img_arr/255.0
    img_arr[0] = np.rot90(img_arr[0], 3)
    img_arr[0] = np.fliplr(img_arr[0])
    img_arr = img_arr.reshape((1, 28, 28, 1))

    result = model.predict_classes([img_arr])
    return chr(emnist_labels[result[0]])

In [10]:
def img_to_str(model: Any, image_file: str):
    letters = letters_extract(image_file)
    s_out = ""
    for i in range(len(letters)):
        dn = letters[i+1][0] - letters[i][0] - letters[i][1] if i < len(letters) - 1 else 0
        s_out += emnist_predict_img(model, letters[i][2])
        if (dn > letters[i][1]/4):
            s_out += ' '
    return s_out

In [11]:
s_out = img_to_str(model, "text.png")
print(s_out)

CUUF5CWUFL
